In [17]:
import cantera as ct
import numpy as np

from matplotlib import pyplot as plt
import csv
import pandas as pd

In [18]:
# input file containing the surface reaction mechanism
cti_file = '../RMG-model/cantera/chem_annotated.cti'

cti_file = '../RMG-model/cantera/chem0050.cti'


gas=ct.Solution(cti_file)
surf = ct.Interface(cti_file,'surface1', [gas])

In [19]:
gas()


  gas:

       temperature             300  K
          pressure          101325  Pa
           density         0.81974  kg/m^3
  mean mol. weight         20.1797  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          1905.6        3.845e+04     J
   internal energy      -1.217e+05       -2.456e+06     J
           entropy          7257.7        1.465e+05     J/K
    Gibbs function     -2.1754e+06        -4.39e+07     J
 heat capacity c_p          1030.1        2.079e+04     J/K
 heat capacity c_v          618.03        1.247e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                Ne              1                1         -17.5994
     [  +38 minor]              0                0



In [20]:
gas.species_names

['Ne',
 'H4N2O2(2)',
 'NH2OH(3)',
 'HNO3(4)',
 'CH3OH(5)',
 'H2O(6)',
 'N2(7)',
 'O2(8)',
 'CO2(9)',
 'H2(10)',
 'CO(11)',
 'C2H6(12)',
 'CH2O(13)',
 'CH3(14)',
 'C3H8(15)',
 'H(16)',
 'C2H5(17)',
 'HCO(18)',
 'CH3CHO(19)',
 'OH(20)',
 'C2H4(21)',
 'CH4(24)',
 'HO2(36)',
 'NH2(82)',
 'HONO(91)',
 'NO2(92)',
 'HNOH(94)',
 'N2H3(99)',
 'H3N2O(193)',
 'H2NO2(195)',
 'S(196)',
 'H2NO3(244)',
 'H2NO3(245)',
 'H2N2O(381)',
 'H2N2O(382)',
 'NNDO(385)',
 'NNO(394)',
 'S(429)',
 'S(489)']

In [21]:
surf.species_names

['X(1)',
 'HX(22)',
 'OX(23)',
 'CH3X(25)',
 'HOX(26)',
 'H2OX(27)',
 'CO2X(28)',
 'OCX(29)',
 'CX(30)',
 'CH2X(31)',
 'CHX(32)',
 'H2NX(211)',
 'SX(214)',
 'H2NOX(216)',
 'H3NOX(217)',
 'HNO3X(220)',
 'CH3OX(222)',
 'CH4OX(223)']



This example solves a plug flow reactor problem, with coupled surface and gas chemistry.





In [37]:
cat_area_per_vol = 100.0 / cm  # Catalyst particle surface area per unit volume. UNKNOW
cat_specific_area = 140 # m2/g
cat_density = 4 / cm**3 # g/m3
cat_area_per_vol = cat_specific_area / cat_density  # m2/m3
cat_area_per_vol # m2/m3

3.5000000000000004e-05

In [45]:
# unit conversion factors to SI
cm = 0.01 # m
minute = 60.0  # s

#######################################################################
# Input Parameters for combustor
#######################################################################
mass_flow_rate =  0.5e-3 # kg/s
temperature_c = 200.0  # Initial Temperature in Celsius
pressure = ct.one_atm # constant
length = 1.1 * cm  # Catalyst bed length. 11mm
area = np.pi * (0.9*cm)**2  # Catalyst bed area.  18mm diameter circle.

### Catalyst properties. Some are hard to estimate
cat_specific_area = 140 # m2/g
cat_density = 4 / cm**3 # g/m3
cat_area_per_vol = cat_specific_area / cat_density  # m2/m3
print(f"Catalyst area per volume {cat_area_per_vol :.2e} m2/m3")
print()

porosity = 0.38  # Catalyst bed porosity (0.38)
# Al2O3 particles are about 0.7mm diameter

output_filename = 'surf_pfr_output.csv'

# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 4001
dt = 1.0

#####################################################################

temperature_kelvin = temperature_c + 273.15  # convert to Kelvin

# import the gas model and set the initial conditions
gas = ct.Solution(cti_file, 'gas')

# should this be mole fractions or mole fractions?
gas.TPY = temperature_kelvin, pressure, 'H4N2O2(2):0.14, NH2OH(3):0.3, HNO3(4):0.3, CH3OH(5):0.16, H2O(6):0.04'

# import the surface model
surf = ct.Interface(cti_file,'surface1', [gas])
surf.TP = temperature_kelvin, ct.one_atm

r_len = length/(NReactors-1)
r_vol = area * r_len * porosity

outfile = open(output_filename,'w')
writer = csv.writer(outfile)
writer.writerow(['Distance (mm)', 'T (C)', 'P (atm)'] +
                gas.species_names + surf.species_names)

# catalyst area in one reactor
cat_area = cat_area_per_vol * r_vol

# Not sure we need the velocity
velocity = mass_flow_rate / (gas.density * area)

# The plug flow reactor is represented by a linear chain of zero-dimensional
# reactors. The gas at the inlet to the first one has the specified inlet
# composition, and for all others the inlet composition is fixed at the
# composition of the reactor immediately upstream. Since in a PFR model there
# is no diffusion, the upstream reactors are not affected by any downstream
# reactors, and therefore the problem may be solved by simply marching from
# the first to last reactor, integrating each one to steady state.

TDY = gas.TDY
cov = surf.coverages

print('    distance       H4N2O2(2)   NH2OH(3)  HNO3(4)   CH3OH(5)')

# create a new reactor
gas.TDY = TDY
r = ct.IdealGasReactor(gas, energy='off')
r.volume = r_vol

# create a reservoir to represent the reactor immediately upstream. Note
# that the gas object is set already to the state of the upstream reactor
upstream = ct.Reservoir(gas, name='upstream')

# create a reservoir for the reactor to exhaust into. The composition of
# this reservoir is irrelevant.
downstream = ct.Reservoir(gas, name='downstream')

# Add the reacting surface to the reactor. The area is set to the desired
# catalyst area in the reactor.
rsurf = ct.ReactorSurface(surf, r, A=cat_area)

# The mass flow rate into the reactor will be fixed by using a
# MassFlowController object.
m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

# We need an outlet to the downstream reservoir. This will determine the
# pressure in the reactor. The value of K will only affect the transient
# pressure difference.
v = ct.PressureController(r, downstream, master=m, K=1e-5)

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 12

# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-21

for n in range(NReactors):
    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = r.thermo.TDY
    upstream.syncState()
    sim.reinitialize()
    sim.advance_to_steady_state()
    dist = n * r_len * 1.0e3   # distance in mm

    if not n % 1:
        print('  {0:10f}  {1:10f}  {2:10f}  {3:10f} {4:10f}'.format(dist, *gas['H4N2O2(2)','NH2OH(3)','HNO3(4)','CH3OH(5)'].X))

    # write the gas mole fractions and surface coverages vs. distance
    writer.writerow([dist, r.T - 273.15, r.thermo.P/ct.one_atm] +
                    list(gas.X) + list(surf.coverages))

outfile.close()
print("Results saved to '{0}'".format(output_filename))


Catalyst area per volume 3.50e-05 m2/m3

    distance       H4N2O2(2)   NH2OH(3)  HNO3(4)   CH3OH(5)
    0.000000    0.016876    0.329293    0.191301   0.200659
    0.002750    0.003136    0.321915    0.188837   0.198076


CanteraError: 
***********************************************************************
CanteraError thrown by CVodesIntegrator::step:
CVodes error encountered. Error code: -27

tout too close to t0 to start integration.
Components with largest weighted error estimates:
40: 0.00387642
34: 0.00248708
22: 0.00190889
55: -0.00174896
31: 0.00160935
28: 0.00104109
39: 0.000983371
4: 0.000838601
36: 0.000664692
27: 0.000646654
***********************************************************************


In [0]:
data = pd.read_csv(output_filename)
data

In [0]:
data['T (C)'].plot()

In [0]:
data[['H4N2O2(2)', 'CH3OH(5)']].plot()

In [0]:
list(data.columns)[:4]

In [0]:
specs = list(data.columns)
specs = specs[4:]
specs

In [0]:
data[specs[1:5]].plot()

for i in range(0,len(specs),10):
    data[specs[i:i+10]].plot()

In [0]:
gas.species('NO(49)').composition

In [0]:
data['NO(49)'].plot()

In [0]:
(data[specs].max()>0.01)

In [0]:
data.loc[0]